In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as matplt
import json
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [2]:

#Ensures we can mount our google drive; this helps us save time uploading files everytime
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Get rough info about the dataset

This is the first general portion whereby we can find out more about the dataset

In [3]:
data= pd.read_json('/content/gdrive/MyDrive/Shopee Code Leagure/contacts.json')

data.info()
data.describe()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Id        500000 non-null  int64 
 1   Email     500000 non-null  object
 2   Phone     500000 non-null  object
 3   Contacts  500000 non-null  int64 
 4   OrderId   500000 non-null  object
dtypes: int64(2), object(3)
memory usage: 19.1+ MB


,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
5,5,,840113148,0,
6,6,hESiBgYTxMnj@hotmail.com,,4,
7,7,hqyunTiaXfIjhO@yahoo.com,,1,vxjYZtMUvNhtEBDruzQfjpsmX
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,
9,9,osIIQgAiZX@hotmail.com,,0,IqewrMXVTJqSsVklWFfOpdRNI


In [4]:
data.loc[3]

Id                                  3
Email              mdllpYmE@gmail.com
Phone                                
Contacts                            0
OrderId     bHquEnCbbsGLqllwryxPsNOxa
Name: 3, dtype: object

In [5]:
#replacing with with NAN
df_new= data.replace("", np.nan)
df_new_order = df_new.copy()
df_new_email = df_new.copy()
df_new_phone = df_new.copy()

In [6]:
df_new_order.dropna(subset = ["OrderId"], inplace=True)
df_new_email.dropna(subset = ["Email"], inplace=True)
df_new_phone.dropna(subset = ["Phone"], inplace=True)

df_order = df_new_order
df_email = df_new_email
df_phone = df_new_phone

df_order

,Id,Email,Phone,Contacts,OrderId
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
7,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX
9,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI
10,10,NaN,918973512631,3,OHRFAXqbAzyxljoUDaOErwtrg
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


In [7]:
OrderList = []
EmailList = []
PhoneList = []

OrderList = df_order.OrderId.unique()
EmailList = df_email.OrderId.unique()
PhoneList = df_phone.OrderId.unique()
OrderList


array(['vDDJJcxfLtSfkooPhbYnJdxov', 'bHquEnCbbsGLqllwryxPsNOxa',
       'vxjYZtMUvNhtEBDruzQfjpsmX', ..., 'JqIXOONvrwvJxZqNxCYHqnrKS',
       'beXCZSzcHaBwAYoDcpQqjuAFO', 'JibSBRgzYdfzkzbTuGUXrcvDX'],
      dtype=object)

In [8]:
duplicatedOrder_Id = df_order[df_order.duplicated(['OrderId'], keep= False)].reset_index()
duplicatedEmail_Id = df_email[df_email.duplicated(['Email'], keep= False)].reset_index()
duplicatedPhone_Id = df_phone[df_phone.duplicated(['Phone'], keep= False)].reset_index()




In [9]:
dup_orderlist =duplicatedOrder_Id["OrderId"].tolist()
dup_Emaillist =duplicatedEmail_Id["Email"].tolist()
dup_Phonelist =duplicatedPhone_Id["Phone"].tolist()

In [10]:
#for testing
def list_duplicates_of(seq,item,data):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return list(data.loc[locs].Id)

'''
source = dup_Phonelist
print(list_duplicates_of(source,'474280608', duplicatedPhone_Id))
'''
def list_duplicates(seq, data):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(data.loc[i].Id)
    return ((key,locs) for key,locs in tally.items() 
                            if len(locs)>1)

def make_list(source, dataframe):
  x= {}
  for i in range(500000):
    x[i]= [int(i)]
  
  for dup in list_duplicates(source, dataframe):
    for i in range(len(dup[1])):
      x[dup[1][i]] = dup[1]
  return (x)



order_trace = make_list(dup_orderlist, duplicatedOrder_Id) 
email_trace = make_list(dup_Emaillist, duplicatedEmail_Id) 
phone_trace = make_list(dup_Phonelist, duplicatedPhone_Id) 



In [11]:
order_trace

{0: [0],
 1: [1, 2458, 476346],
 2: [2],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9, 13, 343161],
 10: [10],
 11: [11],
 12: [12],
 13: [9, 13, 343161],
 14: [14],
 15: [15],
 16: [16],
 17: [17],
 18: [18, 274269, 327057],
 19: [19],
 20: [20],
 21: [21],
 22: [22],
 23: [23, 1944, 49506],
 24: [24],
 25: [25],
 26: [26, 402598],
 27: [27, 246],
 28: [28, 1271, 26068],
 29: [29],
 30: [30],
 31: [31, 66232],
 32: [32],
 33: [33, 459, 499013],
 34: [34],
 35: [35],
 36: [36],
 37: [37, 1864],
 38: [38],
 39: [39],
 40: [40, 1182],
 41: [41],
 42: [42],
 43: [43, 480546],
 44: [44],
 45: [45],
 46: [46],
 47: [47],
 48: [48],
 49: [49],
 50: [50, 226720, 458692],
 51: [51],
 52: [52, 340482],
 53: [53],
 54: [54, 378247],
 55: [55],
 56: [56],
 57: [57],
 58: [58],
 59: [59],
 60: [60, 7964],
 61: [61],
 62: [62],
 63: [63, 471199],
 64: [64],
 65: [65],
 66: [66],
 67: [67],
 68: [68],
 69: [69],
 70: [70],
 71: [71, 335, 2441, 10088, 23134, 169975],
 72: [72],
 73: [

In [12]:
def convert_ticket_trace(some_list):
  ans=""
  for i in some_list:
    ans+= str(i)+"-"

  return ans[:-1]

def contact_val(data,some_list):
  sum_val=0
  for i in some_list:
    sum_val += int(data[i])
  return sum_val
  




In [13]:
def make_finalDf():
  trace_contact = []
  for a in range(len(data)):
    new_list = list(order_trace[a]) + list(phone_trace[a]) + list(email_trace[a])
    new_list= sorted(list(set(new_list)))
    ticket_trace = convert_ticket_trace(new_list)
    ticket_contact = contact_val(data["Contacts"],new_list)
    trace_contact.append( (str(ticket_trace) + ", "+str(ticket_contact)) )
  return trace_contact
  

cool= make_finalDf()




In [14]:
final_df = pd.DataFrame(cool)
final_df


,0
0,"0, 1"
1,"1-2458-476346, 5"
2,"2-348955, 0"
3,"3, 0"
4,"4, 2"
...,...
499995,"499995, 2"
499996,"499996, 4"
499997,"499997, 2"
499998,"121111-499998, 5"


In [17]:

d={'ticket_id': range(len(cool))}
df= pd.DataFrame(data=d)
df["ticket_trace/contact"]= final_df[0]
df

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-476346, 5"
2,2,"2-348955, 0"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


In [19]:
df.to_csv("submission.csv", encoding='utf-8', index= False)